___
## Ciência dos Dados - PROJETO 3 - INSPER

___
#### Freddy Dratwa

#### Victor Vazquez

#### Michel Haggiag

#### Allan Singer

#### Ricardo Ferraro

___

# TEMA AQUI

___

## A. INTRODUÇÃO

## https://www.kaggle.com/jpayne/852k-used-car-listings
## https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html

[Detalhe aqui o objetivo deste projeto!!]

___
## B. MINERANDO DADOS e CARACTERÍSTICAS DO DATASET

[Detalhe aqui as características da base de dados além da análise descritiva.]

In [106]:
import pandas as pd
import os
import numpy as np

### BASE DE DADOS

In [119]:
dataset = pd.read_csv("Dataframe.csv")

In [120]:
if "Unnamed: 0" in dataset:
    dataset = dataset.drop(columns=["Unnamed: 0"])
dataset["ESTADO"] = dataset["ESTADO"].apply(lambda x: x.upper())
dataset["CIDADE"] = dataset["CIDADE"].apply(lambda x: x.upper())
dataset["MARCA"] = dataset["MARCA"].apply(lambda x: x.upper())
dataset["MODELO"] = dataset["MODELO"].apply(lambda x: x.upper())

dataset.head()

,ID,PRECO,ANO,KM,CIDADE,ESTADO,MARCA,MODELO
0,454092.0,15899.0,2017.0,41889.6,GARLAND,TX,CHEVROLET,CRUZESEDAN
1,83936.0,3977.0,2009.0,167579.2,PALESTINE,TX,DODGE,AVENGER4DR
2,1017445.0,13285.0,2014.0,25766.4,HOUSTON,TX,NISSAN,SENTRASR
3,403241.0,14450.0,2008.0,104091.2,WATERBURY,CT,INFINITI,G37
4,308030.0,6795.0,2007.0,178300.8,WESTMINSTER,CO,SCION,TC3DR


In [121]:
# dataset["ESTADO"].value_counts()
dataset["ANO"].describe()

count    100000.000000
mean       2013.298270
std           3.420296
min        1997.000000
25%        2012.000000
50%        2014.000000
75%        2016.000000
max        2018.000000
Name: ANO, dtype: float64

### ANÁLISE DESCRITIVA

___
## C. MODELOS DE PREDIÇÃO

o MODELO DE PREDIÇÃO PELA MÉDIA (Sem uso de variável explicativa).

o MODELO DOS K VIZINHOS MAIS PRÓXIMOS (K-Nearest Neighbors Regression)

o MODELO DE REGRESSÃO LINEAR (Multiple Linear Regression)

o MODELO DE ÁRVORES DE REGRESSÃO (Decision Tree Regression)

___
## D. PROCESSO E ESTATÍSTICAS DE VALIDAÇÃO

[Esse item depende dos resultados das modelagens anteriores! Organize-os aqui de forma clara!]

___
## E. CONCLUSÃO

___
## F. REFERÊNCIAS BIBLIOGRÁFICAS